<h1 align="center">Launch Sites Locations Analysis with Folium</h1>

<div style="text-align:center">
    <img src='https://i.ibb.co/0Y0RVrS/Space-X-Falcon-9-GPS-III-SV01.jpg'>
</div>

The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.

## Import Libraries and Define Auxiliary Functions

In [13]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon
from math import sin, cos, sqrt, atan2, radians

## Mark all launch sites on a map

First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name **spacex_launch_geo.csv** is an augmented dataset with latitude and longitude added for each site:


In [14]:
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
spacex_df=pd.read_csv(URL)
spacex_df.sample(3)

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
32,46,2017-12-23,1:27:00,F9 FT B1036.2,VAFB SLC-4E,Iridium NEXT 4,9600.0,Polar LEO,Iridium Communications,Controlled (ocean),0,34.632834,-120.610745
20,23,2016-04-08,20:43:00,F9 FT B1021.1,CCAFS LC-40,SpaceX CRS-8,3136.0,LEO (ISS),NASA (CRS),Success (drone ship),1,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356


Now, we can take a look at what are the coordinates for each site:

In [15]:
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give us any intuitive insights about where are those launch sites. If you are very good at geography, we can interpret those numbers directly in our mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.

We first need to create a folium **Map** object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.

In [16]:
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use **folium.Circle** to add a highlighted circle area with a text label on a specific coordinate. For example:

In [17]:
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
marker = folium.map.Marker(
    nasa_coordinate,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 18; color:#000000;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

Now, let's add a circle for each launch site in data frame **launch_sites**:

In [18]:
center= (launch_sites_df['Lat'].mean(),launch_sites_df['Long'].mean())
center

(30.08289697, -90.602954325)

In [19]:
site_map = folium.Map(location=center, zoom_start=4.5)

for index,site in launch_sites_df.iterrows():
    circle = folium.Circle([site['Lat'],site['Long']], radius=2000, color='#d35400', fill=True).add_child(folium.Popup(site['Launch Site']))
    marker = folium.map.Marker([site['Lat'],site['Long']],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 18; color:#000000;"><b>%s</b></div>' % site['Launch Site'],
        )
    )
    site_map.add_child(circle)
    site_map.add_child(marker)

site_map

We can observe that the launch sites are relatively near the Equator and close to the coast. These locations seek to maximize the boost given by the Earth's rotational speed and minimize the effects if the launch fails.

## Mark the success/failed launches for each site on the map

Next, let's enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.

In [20]:
spacex_df_success=spacex_df[['Launch Site','Lat','Long','class']]
spacex_df_success.tail()

,Launch Site,Lat,Long,class
51,CCAFS SLC-40,28.563197,-80.57682,0
52,CCAFS SLC-40,28.563197,-80.57682,0
53,CCAFS SLC-40,28.563197,-80.57682,0
54,CCAFS SLC-40,28.563197,-80.57682,1
55,CCAFS SLC-40,28.563197,-80.57682,0


In [21]:
marker_cluster = MarkerCluster()
marker_cluster

In [22]:
marker_color = []
color=''

for index,element in spacex_df_success.iterrows():
    if element['class'] == 0:
        color = 'red'
    else:
        color = 'green'
    marker_color.append(color)

spacex_df_s_f['marker_color']=marker_color

spacex_df_s_f.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


In [23]:
site_map.add_child(marker_cluster)

for index, record in spacex_df_s_f.iterrows():
    marker = folium.Marker([record['Lat'], record['Long']], 
                  icon=folium.Icon(color='white', icon_color=record['marker_color']))
    marker_cluster.add_child(marker)

site_map

## Calculate the distances between a launch site to its proximities

Next, we need to explore and analyze the proximities of launch sites adding mouse position.

In [24]:
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [25]:
def calculate_distance(lat1, lon1, lat2, lon2):
    
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

Calculate the distance between the coastline point and the launch site:

In [26]:
lat1 = 28.573
lon1 = -80.64587

lat2 = 28.57618
lon2 = -80.60771

distance_coastline = calculate_distance(lat1, lon1, lat2, lon2)

distance_coastline

3.7442706510376853

Display the distance between coastline point and launch site using the icon property

In [27]:
closest_coastline= [lat2,lon2]
# for example
distance_marker = folium.Marker(closest_coastline,
                icon=DivIcon(icon_size=(20,20),
                icon_anchor=(0,0),html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
        )
    )
site_map.add_child(distance_marker)


circleclosest_coastline = folium.Circle(
    closest_coastline, radius=100, color='#d35400', fill=True).add_child(folium.Popup('closest_point'))

site_map.add_child(circleclosest_coastline)

Draw a **PolyLine** between a launch site to the selected coastline point:

In [28]:
initial_point= [lat1,lon1]


lines=folium.PolyLine(locations=[initial_point,closest_coastline], weight=2 ,
    color="#0022FF", tooltip='Closest point to coast line')
site_map.add_child(lines)

## Draw a line between a launch site to its closest city, railway, highway, etc.

Create a marker with distance to a closest city, railway, highway, etc.:

In [29]:
city_coordenate= [28.40167,-80.60459]
rail_coordenate= [28.57392,-80.65749]
highway_coordenate=[28.52615,-80.69321]

initial_point= [lat1,lon1]

In [30]:
def line_function(initial_point,final_coordenate):
    line=folium.PolyLine(locations=[initial_point,final_coordenate], weight=2 ,
                          color="#C143FF", tooltip='Closest point to coast line')
    site_map.add_child(line)

    return line

Draw a line between the marker to the launch site:

In [31]:
city_line=line_function(initial_point,city_coordenate)
rail_line=line_function(initial_point,rail_coordenate)
highway_line=line_function(initial_point,highway_coordenate)

site_map

*   Are launch sites in close proximity to railways? Yes
*   Are launch sites in close proximity to highways? Yes
*   Are launch sites in close proximity to coastline? Yes
*   Do launch sites keep certain distance away from cities? No

The launch sites are close to railways and highways for transporting machines, equipment, tools, and other supplies quickly. The sites are close to the coastline and far from cities because the probability of failure shouldn't affect the public.